In [73]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [74]:
!pip install -r "/content/drive/MyDrive/Colab Notebooks/6S Consulting /requirements[1].txt"

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-14' coro=<Server.serve() done, defined at /usr/local/lib/python3.11/dist-packages/uvicorn/server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-3773281834.py", line 9, in <cell line: 0>
    start_server()
  File "/tmp/ipython-input-3773281834.py", line 6, in start_server
    server.run()
  File "/usr/local/lib/python3.11/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/nest_asyncio.py

In [75]:
!pip install pyngrok

In [76]:
from typing import  List
from fastapi import FastAPI
import os
from fastapi.middleware.cors import CORSMiddleware
import requests
from pathlib import Path as p
import ast
import uvicorn
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableWithMessageHistory
from pydantic import BaseModel
import pymupdf as fitz  # PyMuPDF for PDF handling
import shutil
from datetime import datetime, timedelta
from langchain_core.chat_history import InMemoryChatMessageHistory
import nest_asyncio
from pyngrok import ngrok
import uvicorn

In [77]:
from google.colab import userdata
geminiAPI = userdata.get('GOOGLE_API_KEY')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [78]:
llm1 = ChatGoogleGenerativeAI(api_key=geminiAPI, model='gemini-2.0-flash')

In [79]:
# Request Body for Endpoints
class File(BaseModel):
    files: List[str]
    rewrite: bool

In [80]:
class FileRequest(BaseModel):
    userid: str
    query: str

In [81]:
# Intializing Fast API
app = FastAPI()

origins = [
    "*"
]

app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [82]:
def get_system_prompt():
    return """
    You are an advanced AI assistant. Your primary goal is to understand the user's intent, context, and preferences from both current input and prior conversation history. Always provide clear, relevant, and helpful responses that align with the user’s objectives.

    Guidelines:
    1. **Context Awareness**: Retain and use conversation history, background details, and user-specific data to keep responses coherent and personalized.
    2. **Input Understanding**: Interpret user inputs even if they are brief, ambiguous, or informal. Ask clarifying questions when needed.
    3. **Tone & Style**: Be professional, concise, and adaptive. Match the user's communication style (formal, casual, technical, etc.).
    4. **Answer Quality**: Provide factually accurate, well-structured, and actionable answers. Where uncertainty exists, explain possible options instead of fabricating information.
    5. **Proactivity**: Anticipate user needs, suggest next steps, and provide examples or templates when useful.
    6. **Boundaries**: Refuse unsafe, harmful, or irrelevant requests politely. Stay aligned with ethical and responsible AI use.

    Your role is not only to answer but to **understand, contextualize, and guide** the user effectively.

    """

def get_human_prompt():
    return """
    Use the following context about the organization to answer the question.
    Context: {context}
    _________________________________________________________________
    Answer the following query in a conversational way in the same language:
    {query}
    """

In [83]:
chat_history = {}

def get_chat_history(session_id : str) -> InMemoryChatMessageHistory() :
  if session_id not in chat_history :
    chat_history[session_id] = InMemoryChatMessageHistory()
  return chat_history[session_id]

In [84]:
# First we will be creating
def get_conversation_chain(vectorstore, context, query,uid):
    retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})
    system_prompt = get_system_prompt()
    human_prompt = get_human_prompt()
    query_prompt = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template("""Your task is to create a retrieval query that will be used to fetch relevant document from the vectordatabase. Identify the users query and generate appropriate retrieval query"""),
        MessagesPlaceholder(variable_name="history"),
        HumanMessagePromptTemplate.from_template("""{query}""")
    ])

    query_chain = {"query": lambda x : x["query"], "history": lambda x : x["history"]} | query_prompt | llm1 | StrOutputParser()
    chat_his = get_chat_history(uid)
    query = query_chain.invoke({"query":query,"history" : chat_his.messages})

    prompt_template = ChatPromptTemplate.from_messages([
        SystemMessagePromptTemplate.from_template(system_prompt),
        MessagesPlaceholder(variable_name="history"),
        HumanMessagePromptTemplate.from_template(human_prompt)
    ])

    documents = retriever.invoke(query)
    context = " ".join([doc.page_content for doc in documents])

    # Create chain - The key fix is here
    chain = prompt_template | llm1 | StrOutputParser()

    # RunnableWithMessageHistory will automatically handle the history
    chat = RunnableWithMessageHistory(
        chain,
        get_session_history=get_chat_history,
        history_messages_key="history",
        input_messages_key="query"  # This should match your input key
    )

    return chat, context

In [85]:
def create_vectorstore(text_chunks, rewrite, uid):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

    if rewrite:
        vectorstore = FAISS.from_texts(text_chunks, embeddings)
        vectorstore.save_local(f"{uid}-vectorstore.json")
    else:
        vectorstore = FAISS.load_local(f"{uid}-vectorstore.json", embeddings, allow_dangerous_deserialization=True)
        vectorstore.add_texts(text_chunks)
        vectorstore.save_local(f"{uid}-vectorstore.json")

    return vectorstore

In [86]:
def get_pdf_text(pdf_urls):
    combined_text = ""
    for url in pdf_urls:
        response = requests.get(url)
        if response.status_code == 200:
            pdf_data = response.content
            with fitz.open(stream=pdf_data, filetype="pdf") as pdf_document:
                for page_num in range(pdf_document.page_count):
                    page = pdf_document[page_num]
                    combined_text += page.get_text()
        else:
            print(f"Failed to fetch PDF from {url}")
    return combined_text

In [87]:
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=8000,
        chunk_overlap=1000,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [88]:
def remove_folder(folder_path):
  try:
      shutil.rmtree(folder_path)
      print(f"Directory '{folder_path}' and all its contents have been removed.")
  except FileNotFoundError:
      print(f"Directory '{folder_path}' does not exist.")
  except Exception as e:
      print(f"Error: {e}")

In [89]:
def create_folder(folder_path, files):
    os.makedirs(folder_path, exist_ok=True)

    for file_url in files:
        try:
            filename = file_url.split("?")[0].split("/")[-1]
            file_path = os.path.join(folder_path, filename)

            with requests.get(file_url, stream=True) as response:
                response.raise_for_status()
                with open(file_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        if chunk:
                            f.write(chunk)

            print(f"Downloaded: {filename}")
        except Exception as e:
            print(f"Error downloading {file_url}: {e}")

In [90]:
@app.post("/process")
def process_files(request:File):
  uid = "Kishore_M"
  files = request.files
  rewrite = request.rewrite
  create_folder(f"{uid}-files",files)
  raw_text = get_pdf_text(files)
  remove_folder(f"{uid}-files")
  text_chunks = get_text_chunks(raw_text)
  vectorstore = create_vectorstore(text_chunks,rewrite,uid)
  return {"message":"Sucessful"}

In [91]:
@app.post("/generate")
def generate_response(request: FileRequest):
    uid = "Kishore_M"
    query = request.query
    context = ()
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
    vectorstore = FAISS.load_local(f"{uid}-vectorstore.json", embeddings, allow_dangerous_deserialization=True)

      # Check if vectorstore was successfully loaded
    if vectorstore is None:
      return {"response": "No vector database found. Please upload documents first."}

    chain,context= get_conversation_chain(vectorstore,context,query,uid)
    result = chain.invoke({"query":query,"context":context},config={"session_id": uid})
    return {"response": result}

In [92]:
nest_asyncio.apply()
ngrok_token = userdata.get('NGROK_AUTHTOKEN')
ngrok.set_auth_token(ngrok_token)

In [93]:
def start_server():
    public_url = ngrok.connect(8000)
    print(f"Public URL: {public_url}")
    config = uvicorn.Config(app=app, host="0.0.0.0", port=8000, log_level="info")
    server = uvicorn.Server(config)
    server.run()

# Start the server
start_server()

Public URL: NgrokTunnel: "https://d43a3d55cdba.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [537]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [537]


KeyboardInterrupt: 